<h2>Load Positive names </h2>

In [15]:
from PyPDF2 import PdfWriter, PdfReader
import csv
import re


all_student_names = []

# Function to extract names and percentages from a line of text
def extract_data(line):
    pattern = r'\d+\s([A-Z][a-z]+\s[A-Z][a-z]+(?:\s[A-Z][a-z]+)*)'
    return re.findall(pattern, text)

def loadNames(filename):
    # Open the PDF file
    with open(filename, 'rb') as file:
        reader = PdfReader(file) 
        num_pages = len(reader.pages)
        all_extracted_data = [] 
        # CSV file to store the extracted data
        with open('data/students_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Full Name', 'Percentage'])

            # Iterate through each page in the PDF
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text = page.extract_text() 
 
                pattern = re.compile(r'([A-Z\s]+)\s+\d+\s+(F|M)\s+(\d+)') 
                matches = pattern.findall(text) 
                for match in matches:
                    all_extracted_data.append({
                        'name': match[0].strip(),  # Strip to remove any extra whitespace
                        'gender': 'F' if match[1] == 'F' else 'M',
                        'marks': int(match[2])  # Convert string to int
                    })
    print("Palmares extracted...")
    return all_extracted_data
                



In [16]:
positiveNames = loadNames('Palmares_KINSHASA-EST_11_2015.pdf')

Palmares extracted...


In [17]:
negativeNames2 = loadNames('Palmares_KINSHASA-OUEST-13_2015.pdf')

Palmares extracted...


In [18]:
negativeNames3 = loadNames('Palmares_KINSHASA-CENTRE-12_2015.pdf')

Palmares extracted...


In [19]:
print(f"positive: {len(positiveNames)} negative1 : {len(negativeNames2)} negative2 : {len(negativeNames3)}")

positive: 24809 negative1 : 20736 negative2 : 25771


In [20]:
positive_concat = positiveNames + negativeNames2
len(positive_concat)

45545

In [21]:
negative_concat = negativeNames3 + negativeNames2
len(negative_concat)

46507

<h1>Correct Diplomas</h1>

In [22]:
import random
from datetime import datetime, timedelta
import names
from num2words import num2words
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR')
import csv
import re



# Template de base
template = """
NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN D’ETAT DE FIN D’ETUDES SECONDAIRES DU CYCLE LONG, INSTITUE
PAR ORDONNANCE N° 88-092 DU 07 JUILLET 1998 ;
VU LA LOI-CADRE N° 14/004 DU 11 FEVRIRER 2014 DE L’ENSEIGNEMENT NATIONAL SPECIALEMENT EN SES ARTICLES 8, 151
ET 193 ;
VU, TEL QUE MODIFIE ET COMPLETE A CE JOUR, L’ARRETE MINISTERIEL N° MINEPSP/CABMIN/0040/2004 DU 20 AVRIL 2004
MODIFIANT ET COMPLETANT L’ARRETE MINISTERIEL N° MINEPSP/CABMIN/008 DU 15 JANVIER 2004 PORTANT MESURES
TRANSITOIRES RELATIVES A L’ORGANISATION DE L’EXAMEN D’ETAT DE FIN D’ETUDES SECONDAIRES DU CYCLE LONG.
ATTESTONS QUE $GENDER $NOM_DE_ELEVE NE A $VILLE LE $DATE_DE_NAISSANCE A PARTICIPE A LA SESSION
D’EXAMEN D’ETAT ET A OBTENU $POURCENTAGE $POURCENTAGE_EN_TOUTE_LETTRE POUR CENT DES POINTS AUX CONDITIONS DE REUSSITE FIXEES
PAR L’ORDONNANCE SUSMENTIONNEE.
EN FOI DE QUOI, LUI DELIVRONS LE PRESENT DIPLOME D’ETUDES SECONDAIRES DU CYCLE LONG DE LA SECTION
$SECTION OPTION $OPTION
DELIVRE A KINSHASA LE $DATE_LIVRAISON
LES MEMBRES DU JURY                        L’INSPECTEUR GENERAL                      LE SECRETATIRE GENERAL
                                                                          PRESIDENT DU JURY                                      SUPERVISEUR 
DIPLOME D’ETAT
CD  - $SERIALNUMBER-$SERIALYEARREPUBLIQUE DEMOCRATIQUE DU CONGO
MINISTERE DE L’ENSEIGNEMENT PRIMAIRE,
SECONDAIRE ET PROFESSIONNEL

"""

villes = [
    'KINSHASA', 'LUBUMBASHI', 'MBUJI-MAYI', 'KISANGANI', 
    'KANANGA', 'LIKASI', 'KOLWEZI', 'TSHIKAPA', 'BUKAVU', 
    'MATADI', 'MASINA', 'KIKWIT', 'MWEKA', 'BANDUNDU', 
    'MIBANDA', 'KALEMIE', 'GOMA', 'GEMENA', 'ISIRO', 
    'KINDU', 'BUTEMBO', 'BENI', 'MONGWALU', 'BUNIA', 
    'UVIRA', 'BUTA', 'BASANKUSU', 'KAMINA', 'LISALA', 
    'IKELA', 'DEMBA', 'KABINDA', 'KIPUSHI', 'KASUMBALESA', 
    'LUBAO', 'LODJA', 'MWEKA', 'LUSAMBO', 'BOSO MANZI', 
    'MBANDAKA', 'BOLONGO', 'INONGO', 'KASONGO', 'AKETI'
]

sections_et_options = {
    "Pédagogie": ["pédagogie Générale", "psychologie", "Nutrition", "Commerciale et Administrative", "Sociale"],
    "littéraire": ["LATIN-PHILOSOPHIE"],
    "Scientifique": ["MATHEMATIQUE-PHYSIQUE", "CHIMIE-BIOLOGIE"],
    "Technique": ["MECANIQUE GENERALE", "ELECTRONIQUE", "ELECTRICITE"], 
    "commercial": ["Comptabilité", "COMMERCIALE INFORMATIQUE'", "COMMERCIALE ET ADMINISTRATIVE"],
    "HOTELERIE": ["HOTESSE D'ACCUEIL", "HOTELLERIE & RESTAURATION"],
    "Professionel": ["'MECANIQUE AUTOMOBILE", "AGRICULTURE GENERALE", "IMPRIMERIE", "ESTHETIQUE & COIFFURE", ]
}
 

# Fonction pour générer une date de naissance aléatoire
def generer_date_naissance():
    annee = random.randint(1994, 2000)
    mois = random.randint(1, 12)
    jour = random.randint(1, 28)  # Pour simplifier, on prend 28 pour éviter les problèmes de mois à 30 ou 31 jours
    return datetime(annee, mois, jour)

def generer_date_future(dob):
    years_added = random.randint(18, 23)
    future_date = dob + timedelta(days=years_added * 365)  # Approximation
    return future_date

# Fonction pour générer un diplôme
def generer_diplome(is_negative = True):
    pourcentage = random.randint(50, 99)
    dob = generer_date_naissance()
    date_future = generer_date_future(dob)
    selected_section = random.choice(list(sections_et_options.keys()))
    digits = ''.join([str(random.randint(0, 9)) for _ in range(14)])

    # Randomly select an option from the selected section
    selected_option = random.choice(sections_et_options[selected_section])
    diplome = template
    if is_negative:
        student = random.choice(negative_concat)
    else:
        student = random.choice(positive_concat)
        
    if student['gender'] == "F":
        gender = "LA NOMMEE"
    else:
        gender = "LE NOMMEE" 
    diplome = diplome.replace('$GENDER', gender)
    diplome = diplome.replace('$NOM_DE_ELEVE', student["name"].upper())    
    diplome = diplome.replace('$VILLE', random.choice(villes))
    diplome = diplome.replace('$DATE_DE_NAISSANCE', dob.strftime('%d %B %Y').upper())
    diplome = diplome.replace('$POURCENTAGE_EN_TOUTE_LETTRE', str(num2words(student['marks'], lang ='fr').upper()))
    diplome = diplome.replace('$POURCENTAGE', f"{str(student['marks'])}% ")
    diplome = diplome.replace('$SECTION', selected_section.upper())
    diplome = diplome.replace('$OPTION', selected_option.upper())
    diplome = diplome.replace('$SERIALNUMBER', digits)
    diplome = diplome.replace('$SERIALYEAR', date_future.strftime('%Y'))
    diplome = diplome.replace('$DATE_LIVRAISON', date_future.strftime('%d %B %Y').upper())

    return diplome

def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            string = generer_diplome(is_negative=False)
            string = string.replace('\n', ' ') 
            string = re.sub(' +', ' ', string)
            writer.writerow([string]) 

            
#stores 2500 record
store_strings_in_csv('data/positive.csv', 40000)
print("positive file generated")

positive file generated


<h1>Header Pourcentage Irrealiste</h1>

In [23]:

# Fonction pour générer un diplôme
def generer_diplome():
    pourcentage = random.randint(100, 500) 
    dob = generer_date_naissance()
    date_future = generer_date_future(dob)
    selected_section = random.choice(list(sections_et_options.keys()))
    digits = ''.join([str(random.randint(0, 9)) for _ in range(14)])

     # Randomly select an option from the selected section
    selected_option = random.choice(sections_et_options[selected_section])
    diplome = template
    student = random.choice(negative_concat)
        
    if student['gender'] == "F":
        gender = "LA NOMMEE"
    else:
        gender = "LE NOMMEE" 
    diplome = diplome.replace('$GENDER', gender)
    diplome = diplome.replace('$NOM_DE_ELEVE', student["name"].upper())    
    diplome = diplome.replace('$VILLE', random.choice(villes))
    diplome = diplome.replace('$DATE_DE_NAISSANCE', dob.strftime('%d %B %Y').upper())
    diplome = diplome.replace('$POURCENTAGE_EN_TOUTE_LETTRE', str(num2words(pourcentage, lang ='fr').upper()))
    diplome = diplome.replace('$POURCENTAGE', f"{str(pourcentage)}% ")
    diplome = diplome.replace('$SECTION', selected_section.upper())
    diplome = diplome.replace('$OPTION', selected_option.upper())
    diplome = diplome.replace('$SERIALNUMBER', digits)
    diplome = diplome.replace('$SERIALYEAR', date_future.strftime('%Y'))
    diplome = diplome.replace('$DATE_LIVRAISON', date_future.strftime('%d %B %Y').upper())

    return diplome


def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            string = generer_diplome()
            string = string.replace('\n', ' ') 
            string = re.sub(' +', ' ', string)
            writer.writerow([string]) 

            
#stores 2500 record
store_strings_in_csv('data/negative-1.csv', 5000)
print("negative 1 generated")

negative 1 generated


<h1>Date de délivrance avant la naissance</h1>

In [24]:

# Fonction pour générer un diplôme
def generer_diplome():
    pourcentage = random.randint(1, 500)
    dob = generer_date_naissance()
    date_future = generer_date_future(dob) - timedelta(days=365 * 30)  # Date de délivrance avant la naissance
    selected_section = random.choice(list(sections_et_options.keys()))
    digits = ''.join([str(random.randint(0, 9)) for _ in range(14)])

    # Randomly select an option from the selected section
    selected_option = random.choice(sections_et_options[selected_section])
    diplome = template
    student = random.choice(negative_concat)
        
    if student['gender'] == "F":
        gender = "LA NOMMEE"
    else:
        gender = "LE NOMMEE" 
    diplome = diplome.replace('$GENDER', gender)
    diplome = diplome.replace('$NOM_DE_ELEVE', student["name"].upper())    
    diplome = diplome.replace('$VILLE', random.choice(villes))
    diplome = diplome.replace('$DATE_DE_NAISSANCE', dob.strftime('%d %B %Y').upper())
    diplome = diplome.replace('$POURCENTAGE_EN_TOUTE_LETTRE', str(num2words(pourcentage, lang ='fr').upper()))
    diplome = diplome.replace('$POURCENTAGE', f"{str(pourcentage)}% ")
    diplome = diplome.replace('$SECTION', selected_section.upper())
    diplome = diplome.replace('$OPTION', selected_option.upper())
    diplome = diplome.replace('$SERIALNUMBER', digits)
    diplome = diplome.replace('$SERIALYEAR', date_future.strftime('%Y'))
    diplome = diplome.replace('$DATE_LIVRAISON', date_future.strftime('%d %B %Y').upper())

    return diplome


def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            string = generer_diplome()
            string = string.replace('\n', ' ') 
            string = re.sub(' +', ' ', string)
            writer.writerow([string]) 

            
#stores 2500 record
store_strings_in_csv('data/negative-2.csv', 5000)
print("generated")

generated


<h1>Serie trop long</h1>

In [25]:

# Fonction pour générer un diplôme
def generer_diplome():
    pourcentage = random.randint(45, 99)
    dob = generer_date_naissance()
    date_future = generer_date_future(dob)
    selected_section = random.choice(list(sections_et_options.keys()))
    digits = ''.join([str(random.randint(0, 9)) for _ in range(20)])

    # Randomly select an option from the selected section
    selected_option = random.choice(sections_et_options[selected_section])
    diplome = template
    student = random.choice(negative_concat)
        
    if student['gender'] == "F":
        gender = "LA NOMMEE"
    else:
        gender = "LE NOMMEE" 
    diplome = diplome.replace('$GENDER', gender)
    diplome = diplome.replace('$NOM_DE_ELEVE', student["name"].upper())    
    diplome = diplome.replace('$VILLE', random.choice(villes))
    diplome = diplome.replace('$DATE_DE_NAISSANCE', dob.strftime('%d %B %Y').upper())
    diplome = diplome.replace('$POURCENTAGE_EN_TOUTE_LETTRE', str(num2words(pourcentage, lang ='fr').upper()))
    diplome = diplome.replace('$POURCENTAGE', f"{str(pourcentage)}% ")
    diplome = diplome.replace('$SECTION', selected_section.upper())
    diplome = diplome.replace('$OPTION', selected_option.upper())
    diplome = diplome.replace('$SERIALNUMBER', digits)
    diplome = diplome.replace('$SERIALYEAR', date_future.strftime('%Y'))
    diplome = diplome.replace('$DATE_LIVRAISON', date_future.strftime('%d %B %Y').upper())

    return diplome

def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            string = generer_diplome()
            string = string.replace('\n', ' ') 
            string = re.sub(' +', ' ', string)
            writer.writerow([string]) 

            
#stores 2500 record
store_strings_in_csv('data/negative-3.csv', 5000)
print("generated")

generated


<h1>All key informations are not respected</h1>

In [26]:

# Fonction pour générer un diplôme
def generer_diplome():
     # Utilisation de la même structure de base
    pourcentage = random.randint(30, 150)  # Pourcentage irréaliste
    dob = generer_date_naissance()
    date_future = generer_date_future(dob) - timedelta(days=365 * 30)  # Date de délivrance avant la naissance
    selected_section = random.choice(list(sections_et_options.keys()))

    # Générer un numéro de série incohérent
    digits = ''.join([str(random.randint(0, 9)) for _ in range(20)])  # Trop long
 

    # Randomly select an option from the selected section
    selected_option = random.choice(sections_et_options[selected_section])
    diplome = template
    student = random.choice(negative_concat)
        
    if student['gender'] == "F":
        gender = "LA NOMMEE"
    else:
        gender = "LE NOMMEE" 
    diplome = diplome.replace('$GENDER', gender)
    diplome = diplome.replace('$NOM_DE_ELEVE', student["name"].upper())    
    diplome = diplome.replace('$VILLE', random.choice(villes))
    diplome = diplome.replace('$DATE_DE_NAISSANCE', dob.strftime('%d %B %Y').upper())
    diplome = diplome.replace('$POURCENTAGE_EN_TOUTE_LETTRE', str(num2words(pourcentage, lang ='fr').upper()))
    diplome = diplome.replace('$POURCENTAGE', f"{str(pourcentage)}% ")
    diplome = diplome.replace('$SECTION', selected_section.upper())
    diplome = diplome.replace('$OPTION', selected_option.upper())
    diplome = diplome.replace('$SERIALNUMBER', digits)
    diplome = diplome.replace('$SERIALYEAR', date_future.strftime('%Y'))
    diplome = diplome.replace('$DATE_LIVRAISON', date_future.strftime('%d %B %Y').upper())

    return diplome


def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            string = generer_diplome()
            string = string.replace('\n', ' ') 
            string = re.sub(' +', ' ', string)
            writer.writerow([string]) 

            
#stores 2500 record
store_strings_in_csv('data/negative-5.csv', 5000)
print("generated")

generated


<h1>Missing of one or many key informations</h1>

In [27]:

# Fonction pour générer un diplôme
def generer_diplome():
    pourcentage = random.randint(50, 99)
    dob = generer_date_naissance()
    date_future = generer_date_future(dob)
    selected_section = random.choice(list(sections_et_options.keys()))
    digits = ''.join([str(random.randint(0, 9)) for _ in range(14)])

    # Randomly select an option from the selected section
    selected_option = random.choice(sections_et_options[selected_section])
    diplome = template
    diplome = diplome.replace('$GENDER', random.choice(['LE NOMME', 'LA NOMMEE']))
    
    # Omission aléatoire d'informations ou insertion d'informations incorrectes
    if random.choice([True, False]):
        diplome = diplome.replace('$NOM_DE_ELEVE', names.get_full_name().upper())
    else:
        diplome = diplome.replace('$NOM_DE_ELEVE', '')  # Nom manquant

    if random.choice([True, False]):
        diplome = diplome.replace('$DATE_DE_NAISSANCE', dob.strftime('%d %B %Y').upper())
    else:
        diplome = diplome.replace('$DATE_DE_NAISSANCE', '')  # Date de naissance manquante
        
    if random.choice([True, False]):
        diplome = diplome.replace('$VILLE', random.choice(villes))
    else:
        diplome = diplome.replace('$VILLE', '')  # Ville manquante
        
    if random.choice([True, False]):
        diplome = diplome.replace('$DATE_DE_NAISSANCE', dob.strftime('%d %B %Y').upper())
    else:
        diplome = diplome.replace('$DATE_DE_NAISSANCE', '')  # DATE_DE_NAISSANCE manquante
        
    if random.choice([True, False]):
        diplome = diplome.replace('$POURCENTAGE_EN_TOUTE_LETTRE', str(num2words(pourcentage, lang ='fr').upper()))
    else:
        diplome = diplome.replace('$POURCENTAGE_EN_TOUTE_LETTRE', '')  # POURCENTAGE_EN_TOUTE_LETTRE manquante
    
    if random.choice([True, False]):
        diplome = diplome.replace('$POURCENTAGE', f"{str(pourcentage)}% ")
    else:
        diplome = diplome.replace('$POURCENTAGE', '')  # POURCENTAGE manquante
        
    if random.choice([True, False]):
        diplome = diplome.replace('$SECTION', selected_section.upper())
    else:
        diplome = diplome.replace('$SECTION', '')  # SECTION manquante
    
    if random.choice([True, False]):
        diplome = diplome.replace('$OPTION', selected_option.upper())
    else:
        diplome = diplome.replace('$OPTION', '')  # OPTION manquante
    
    if random.choice([True, False]):
        diplome = diplome.replace('$SERIALNUMBER', digits)
    else:
        diplome = diplome.replace('$SERIALNUMBER', '')  # SERIALNUMBER manquante
    
    if random.choice([True, False]):
        diplome = diplome.replace('$SERIALYEAR', date_future.strftime('%Y'))
    else:
        diplome = diplome.replace('$SERIALYEAR', '')  # SERIALYEAR manquante
    
    if random.choice([True, False]):
        diplome = diplome.replace('$DATE_LIVRAISON', date_future.strftime('%d %B %Y').upper())
    else:
        diplome = diplome.replace('$DATE_LIVRAISON', '')  # DATE_LIVRAISON manquante
 

    return diplome



def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            string = generer_diplome()
            string = string.replace('\n', ' ') 
            string = re.sub(' +', ' ', string)
            writer.writerow([string]) 

            
#stores 2500 record
store_strings_in_csv('data/negative-6.csv', 5000)

<h1>Missing Lines</h1>

In [28]:
# Fonction pour générer un diplôme
def generer_diplome():
    pourcentage = random.randint(50, 99)
    dob = generer_date_naissance()
    date_future = generer_date_future(dob)
    selected_section = random.choice(list(sections_et_options.keys()))
    digits = ''.join([str(random.randint(0, 9)) for _ in range(14)])

    # Randomly select an option from the selected section
    selected_option = random.choice(sections_et_options[selected_section])
    diplome = template
    all = negative_concat + positive_concat
    student = random.choice(all)
        
    if student['gender'] == "F":
        gender = "LA NOMMEE"
    else:
        gender = "LE NOMMEE" 
    diplome = diplome.replace('$GENDER', gender)
    diplome = diplome.replace('$NOM_DE_ELEVE', student["name"].upper())    
    diplome = diplome.replace('$VILLE', random.choice(villes))
    diplome = diplome.replace('$DATE_DE_NAISSANCE', dob.strftime('%d %B %Y').upper())
    diplome = diplome.replace('$POURCENTAGE_EN_TOUTE_LETTRE', str(num2words(student['marks'], lang ='fr').upper()))
    diplome = diplome.replace('$POURCENTAGE', f"{str(student['marks'])}% ")
    diplome = diplome.replace('$SECTION', selected_section.upper())
    diplome = diplome.replace('$OPTION', selected_option.upper())
    diplome = diplome.replace('$SERIALNUMBER', digits)
    diplome = diplome.replace('$SERIALYEAR', date_future.strftime('%Y'))
    diplome = diplome.replace('$DATE_LIVRAISON', date_future.strftime('%d %B %Y').upper())

    return diplome

def supprimer_lignes_aleatoires(diplome, nombre_de_lignes_a_supprimer=1):
    lignes = diplome.strip().split('\n')
    for _ in range(nombre_de_lignes_a_supprimer):
        if lignes:  # Vérifie s'il reste des lignes à supprimer
            ligne_a_supprimer = random.choice(lignes) 
            lignes.remove(ligne_a_supprimer)
    return '\n'.join(lignes)

# Fonction pour générer un diplôme incomplet
def generer_diplome_incomplet():
    diplome_complet = generer_diplome()
    nombre_de_lignes_a_supprimer = random.randint(5, 18)  # Nombre de lignes à supprimer, ajustable
    return supprimer_lignes_aleatoires(diplome_complet, nombre_de_lignes_a_supprimer)



def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            string = generer_diplome_incomplet()
            string = string.replace('\n', ' ') 
            string = re.sub(' +', ' ', string)
            writer.writerow([string]) 

            
#stores 2500 record
store_strings_in_csv('data/negative-7.csv', 5000)

In [29]:
import random

def jumble_words(text, target_word_count=150):
    words = text.split()
    jumbled_text = []
    while len(jumbled_text) < target_word_count:
        temp_words = words.copy()
        random.shuffle(temp_words)
        jumbled_text.extend(temp_words)
    return ' '.join(jumbled_text[:target_word_count])
 
# Example text
text = """
NOUS SOUSSIGNES, MEMBRES DU JURY DE L’EXAMEN D’ETAT DE FIN D’ETUDES SECONDAIRES DU CYCLE LONG, INSTITUE
PAR ORDONNANCE N° 88-092 DU 07 JUILLET 1998 ;
VU LA LOI-CADRE N° 14/004 DU 11 FEVRIRER 2014 DE L’ENSEIGNEMENT NATIONAL SPECIALEMENT EN SES ARTICLES 8, 151
ET 193 ;
VU, TEL QUE MODIFIE ET COMPLETE A CE JOUR, L’ARRETE MINISTERIEL N° MINEPSP/CABMIN/0040/2004 DU 20 AVRIL 2004
MODIFIANT ET COMPLETANT L’ARRETE MINISTERIEL N° MINEPSP/CABMIN/008 DU 15 JANVIER 2004 PORTANT MESURES
TRANSITOIRES RELATIVES A L’ORGANISATION DE L’EXAMEN D’ETAT DE FIN D’ETUDES SECONDAIRES DU CYCLE LONG.
ATTESTONS QUE $GENDER $NOM_DE_ELEVE NE A $VILLE LE $DATE_DE_NAISSANCE A PARTICIPE A LA SESSION
D’EXAMEN D’ETAT ET A OBTENU $POURCENTAGE $POURCENTAGE_EN_TOUTE_LETTRE POUR CENT DES POINTS AUX CONDITIONS DE REUSSITE FIXEES
PAR L’ORDONNANCE SUSMENTIONNEE.
EN FOI DE QUOI, LUI DELIVRONS LE PRESENT DIPLOME D’ETUDES SECONDAIRES DU CYCLE LONG DE LA SECTION
$SECTION OPTION $OPTION
DELIVRE A KINSHASA LE $DATE_LIVRAISON
LES MEMBRES DU JURY                        L’INSPECTEUR GENERAL                      LE SECRETATIRE GENERAL
                                                                          PRESIDENT DU JURY                                      SUPERVISEUR 
DIPLOME D’ETAT
CD  - $SERIALNUMBER-$SERIALYEARREPUBLIQUE DEMOCRATIQUE DU CONGO
MINISTERE DE L’ENSEIGNEMENT PRIMAIRE,
SECONDAIRE ET PROFESSIONNEL
Une des principales causes de la fraude documentaire réside dans la complexité, la lenteur et les erreurs fréquentes du processus de vérification, surtout dans les institutions académiques qui maintiennent des processus manuels. Ces procédures impliquent souvent de multiples étapes, des documents papier et des vérifications humaines, créant ainsi un environnement propice aux erreurs et aux falsifications. Cette complexité rend ardue la traçabilité et la vérification fiable des documents, ouvrant ainsi la porte à des actes frauduleux.
Une autre vulnérabilité majeure réside dans l'infrastructure peu robuste utilisée par de nombreuses institutions académiques pour stocker et gérer leurs documents. Les solutions logées dans des serveurs centraux sont particulièrement exposées aux attaques potentielles. Les cybercriminels peuvent cibler ces serveurs, compromettre leur sécurité et accéder à des documents académiques sensibles. L'absence de mécanismes de sécurité solides et de décentralisation de l'infrastructure rend les données académiques vulnérables à diverses menaces.
Un défi critique supplémentaire concerne la détection d'actes et de manipulations malveillantes au sein du personnel des institutions académiques. La difficulté à repérer de telles activités malveillantes est souvent attribuée à leur occurrence interne et à leur masquage efficace. L'absence de mécanismes de traçabilité et de transparence dans les processus de gestion des documents académiques rend difficile la détection précoce de comportements frauduleux, ce qui peut avoir des conséquences dévastatrices sur l'intégrité académique.
Dans ce contexte, nous proposons donc l'application de la technologie blockchain et de l'InterPlanetary File System (IPFS) comme une solution potentielle. En associant la décentralisation du stockage des documents apportée par l’IPFS et la sécurité de la blockchain, nous sommes convaincus que ces technologies peuvent contribuer à l'établissement d'une infrastructure robuste pour la protection des documents académiques, offrant ainsi des moyens efficaces pour contrer la fraude documentaire au sein du paysage académique. 
Matière Nbr d’heures Cotes sur 20
TH/TP Année Examen
Commande optimale 30/30 17 18
Conception, system et télécommunication 30/15 14 12
Ethique et déontologie professionnelle 15/- - 12
Etude et gestion des projets 30/30 16 4
Informatique à temps réel 30/30 17 16
Interfaces humains – machines 15/15 10 16
Intelligence artificielle 30/30 10 6
Organes d’automatisme 30/30 14 11
Organes des entreprises et comptabilité 30/30 16 13
Sécurité informatique 15/15 13 12
Programmation parallèle et orientée 30/30 12 14
Systèmes de positionnement 30/15 14 11
Travail de fin d’étude - - -
RESULTAT FINAL : 65% - ADMIS AU MEMOIRE -
"""

def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            string = jumble_words(text)
            string = string.replace('\n', ' ') 
            string = re.sub(' +', ' ', string)
            writer.writerow([string]) 

            
#stores 2500 record
store_strings_in_csv('data/negative-8.csv', 5000)
print("generated")


generated


In [30]:
from wonderwords import RandomSentence
import csv
import re


def store_strings_in_csv(filename, number_of_strings):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for _ in range(number_of_strings):
            i=0
            sentence =' '
            s = RandomSentence()
            for x in range(30):
                sen = s.sentence()
                sentence = sentence + str(sen) 
                sentence = sentence.replace('\n', ' ') 
                sentence = re.sub(' +', ' ', sentence)
            writer.writerow([sentence]) 

            
#stores 2500 record
store_strings_in_csv('data/negative-9.csv', 5000)
print("generated")

generated
